In [12]:
import os

# Define the directory containing the log files
log_directory = '/home/patrick_w/sim_loop/results/logs/'

# Get a sorted list of all .txt files in the directory
log_files = sorted([f for f in os.listdir(log_directory) if f.endswith('.txt')])

# Loop through each log file and read its contents
for filename in log_files:
    log_file_path = os.path.join(log_directory, filename)
    
    # Open the log file and read its contents
    with open(log_file_path, 'r') as file:
        log_contents = file.readlines()  # Read lines into a list
    
    # Initialize a dictionary to hold parameters
    parameters = {}
    parameters_started = False
    collision_detected = False  # Flag to check for collision line

    # Loop through the lines to find "6 parameters:" and extract parameters
    for line in log_contents:
        if "6 parameters:" in line:
            parameters_started = True  # Start reading parameters
            continue  # Skip the line with "6 parameters:"
        
        if parameters_started:
            # Split the line into key and value
            if '=' in line:
                key, value = line.split('=')
                key = key.replace(' ', '')
                value = value.replace(' ', '')
                value = value.replace("\n", "")
                parameters[key.strip()] = float(value.strip())  # Store as float
                if key == "EgoStartS":
                    break
        # Check for the collision line
        if "Collision between Ego and OverTaker" in line:
            collision_detected = True

    # Print the parameters extracted from the log file
    print(f"Parameters from {filename}:")
    for key, value in parameters.items():
        print(f"{key}: {value}")
    
    # Print whether the collision was detected
    if collision_detected:
        print("Collision detected: Yes")
    else:
        print("Collision detected: No")

    print("\n" + "="*40 + "\n")  # Separator between log files

Parameters from log.txt:
Collision detected: No


Parameters from log_100_of_104.txt:
v_delta: 16.0
s_delta: 70.0
EgoSpeed: 108.0
EgoStartS: 25.0
Collision detected: No


Parameters from log_101_of_104.txt:
v_delta: 21.0
s_delta: 70.0
EgoSpeed: 108.0
EgoStartS: 25.0
Collision detected: No


Parameters from log_102_of_104.txt:
v_delta: 26.0
s_delta: 70.0
EgoSpeed: 108.0
EgoStartS: 25.0
Collision detected: No


Parameters from log_103_of_104.txt:
v_delta: 31.0
s_delta: 70.0
EgoSpeed: 108.0
EgoStartS: 25.0
Collision detected: No


Parameters from log_104_of_104.txt:
v_delta: 36.0
s_delta: 70.0
EgoSpeed: 108.0
EgoStartS: 25.0
Collision detected: No


Parameters from log_10_of_104.txt:
v_delta: 6.0
s_delta: 15.0
EgoSpeed: 108.0
EgoStartS: 25.0
Collision detected: No


Parameters from log_11_of_104.txt:
v_delta: 11.0
s_delta: 15.0
EgoSpeed: 108.0
EgoStartS: 25.0
Collision detected: No


Parameters from log_12_of_104.txt:
v_delta: 16.0
s_delta: 15.0
EgoSpeed: 108.0
EgoStartS: 25.0
Collision d